# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# EDA

In [2]:
df = pd.read_csv("./train.csv")

In [3]:
df.head()

,Unnamed: 0,id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,0,261358,Y. Sylla,Youssuf Sylla,ST,58,76,525000.0,952.712115,18,...,43,13,19,17,7,6,8,10,7,NaN
1,1,252435,J. Austerfield,Josh Austerfield,"CM, CDM",55,75,350000.0,902.232819,19,...,48,40,48,45,14,8,5,14,11,NaN
2,2,248723,C. Malley,Connor Malley,"CAM, CM",53,66,230000.0,2873.818374,21,...,52,26,23,31,14,8,13,9,6,NaN
3,3,251157,Lee Gwang Yeon,이광연 Gwang Yeon Lee,GK,60,71,475000.0,678.608124,21,...,26,8,14,12,60,60,54,63,61,23.0
4,4,205393,J. Baldwin,Jack Baldwin,CB,63,65,475000.0,1911.674228,28,...,67,62,63,62,11,12,6,11,14,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15391 entries, 0 to 15390
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15391 non-null  int64  
 1   id                           15391 non-null  int64  
 2   short_name                   15391 non-null  object 
 3   long_name                    15391 non-null  object 
 4   player_positions             15391 non-null  object 
 5   overall                      15391 non-null  int64  
 6   potential                    15391 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15342 non-null  float64
 9   age                          15391 non-null  int64  
 10  dob                          15391 non-null  object 
 11  height_cm                    15391 non-null  int64  
 12  weight_kg                    15391 non-null  int64  
 13  club_team_id    

# Data Cleaning

In [5]:
# First we remove all data points that have no value in the dataset because this is
# the main thing we are predicting and we cannot work with data that does not have this value
df.dropna(subset=['value_eur'], inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [6]:
# Next we will handle the 'club_loaned_from' column, we will convert this to a
# 'is_on_loan' which is 0 for no and 1 for yes
df['is_on_loan'] = df['club_loaned_from'].notnull().astype(int)

# Drop the club_loaned_from column
df = df.drop('club_loaned_from', axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [7]:
# Next we will do the same for the national team where if there is a national jersey number
# we set that they have played for the national team and if no number exists then they did not play
df['played_for_nation'] = df['nation_jersey_number'].notnull().astype(int)

# Drop the club_loaned_from column
df = df.drop('nation_jersey_number', axis=1)
df = df.drop('nation_position', axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [8]:
# Next we shall fix the club_position column, it has values such as RES, SUB and various positions
# We shall change it to be RES = 3, SUB  = 2 and the rest = 1 this is because reserves are worst then subs are second best
# and the remaining are started which are the best since there is meaning in the number differences this is best
mapping = {'RES': 3, 'SUB': 2}
df['club_position'] = df['club_position'].map(mapping).fillna(1).astype(int)

In [9]:
df['club_position'].tail()

15386    2
15387    3
15388    1
15389    3
15390    1
Name: club_position, dtype: int64

In [10]:
df['club_joined'].head()

0    2020-09-01
1    2019-08-01
2    2018-07-01
3    2019-01-04
4    2021-08-25
Name: club_joined, dtype: object

In [11]:
# Next is the club_joined column we will find the difference and use that to get the number
# of years at club and for those that are null it means they were at the club from the
# start, so for this we will take the max of the database to get the longest number of years someone
# has been at a club and use this for all 

# Ensure the column is in datetime format
df['club_joined'] = pd.to_datetime(df['club_joined'], errors='coerce')

# Extract the year (will be NaN if club_joined was null or invalid)
df['joined_year'] = df['club_joined'].dt.year

# Calculate the number of years at club
df['number_of_years_at_club'] = 2021 - df['joined_year']

# Find the max number of years (ignoring nulls)
max_years = df['number_of_years_at_club'].max()

# Fill nulls with max_years
df['number_of_years_at_club'].fillna(max_years, inplace=True)

# Drop the helper column and original column
df.drop('joined_year', axis=1, inplace=True)
df.drop('club_joined', axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [12]:
df['number_of_years_at_club'].max()

19.0

For the release clause column, we do not want to remove column as it could help predict player value quite well
Instead we will build a model with the release clause values, and another model without the entire release clause column


In [13]:
# We will convert player tags and traits into have traits and tags columns
df['has_tags'] = df['player_tags'].notnull().astype(int)
df['has_traits'] = df['player_traits'].notnull().astype(int)

df.drop('player_tags', axis=1, inplace=True)
df.drop('player_traits', axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [14]:
# Next we shall work on the defending column, we will create a siple linear regression to 
# predict defeding from 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle', 'mentality_interceptions'
# then use that to extrapolate the defending value in the remaining columns
# the columns were chosen based on research
train_df = df[df['defending'].notnull()]
predict_df = df[df['defending'].isnull()]
X_train = train_df[['defending_marking_awareness', 
                    'defending_standing_tackle', 
                    'defending_sliding_tackle', 'mentality_interceptions']]
y_train = train_df['defending']
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_defense = LinearRegression()
model_defense.fit(X_train1, y_train1)
y_pred = model_defense.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Coefficients and intercept (for transparency)
print("MSE:", mse)
print(f"Validation R² Score: {r2:.2f}")
print("Coefficients:", model_defense.coef_)
print("Intercept:", model_defense.intercept_)

# Retrain on ALL non-null data for better accuracy
model_defense = LinearRegression()
model_defense.fit(X_train, y_train)

X_predict = predict_df[['defending_marking_awareness',
                'defending_standing_tackle',
                'defending_sliding_tackle', 'mentality_interceptions']]

# Generate predictions
predicted_defending = model_defense.predict(X_predict)

# Fill missing values in original DataFrame
df.loc[df['defending'].isnull(), 'defending'] = predicted_defending

MSE: 1.3184353865800489
Validation R² Score: 0.99
Coefficients: [0.32262298 0.298127   0.08103073 0.21590816]
Intercept: 4.807193145661245


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [16]:
# Next we shall work on the pace column, we will create a siple linear regression to 
# predict defeding from movement_acceleration and movement_sprint_speed
# then use that to extrapolate the pace value in the remaining columns
# the columns were chosen based on research
train_df = df[df['pace'].notnull()]
predict_df = df[df['pace'].isnull()]
X_train = train_df[['movement_acceleration', 
                    'movement_sprint_speed']]
y_train = train_df['pace']
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_pace = LinearRegression()
model_pace.fit(X_train1, y_train1)
y_pred = model_pace.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Coefficients and intercept (for transparency)
print("MSE:", mse)
print(f"Validation R² Score: {r2:.2f}")
print("Coefficients:", model_pace.coef_)
print("Intercept:", model_pace.intercept_)

# Retrain on ALL non-null data for better accuracy
model_pace = LinearRegression()
model_pace.fit(X_train, y_train)

X_predict = predict_df[['movement_acceleration', 
                    'movement_sprint_speed']]

# Generate predictions
predicted_pace = model_pace.predict(X_predict)

# Fill missing values in original DataFrame
df.loc[df['pace'].isnull(), 'pace'] = predicted_pace

MSE: 0.08392521867652088
Validation R² Score: 1.00
Coefficients: [0.44805689 0.55143434]
Intercept: 0.044253160711633655


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [18]:
# Next we shall work on the shooting column, we will create a siple linear regression to 
# predict defeding from attacking_finishing, attacking_volleys, power_shot_power, power_long_shots, mentality_positioning and mentality_penalties
# then use that to extrapolate the shooting value in the remaining columns
# the columns were chosen based on research
train_df = df[df['shooting'].notnull()]
predict_df = df[df['shooting'].isnull()]
X_train = train_df[['attacking_finishing', 
                    'attacking_volleys', 'power_shot_power', 'power_long_shots', 'mentality_positioning', 'mentality_penalties']]
y_train = train_df['shooting']
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_shooting = LinearRegression()
model_shooting.fit(X_train1, y_train1)
y_pred = model_shooting.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Coefficients and intercept (for transparency)
print("MSE:", mse)
print(f"Validation R² Score: {r2:.2f}")
print("Coefficients:", model_shooting.coef_)
print("Intercept:", model_shooting.intercept_)

# Retrain on ALL non-null data for better accuracy
model_shooting = LinearRegression()
model_shooting.fit(X_train, y_train)

X_predict = predict_df[['attacking_finishing', 
                    'attacking_volleys', 'power_shot_power', 'power_long_shots', 'mentality_positioning', 'mentality_penalties']]

# Generate predictions
predicted_shooting = model_shooting.predict(X_predict)

# Fill missing values in original DataFrame
df.loc[df['shooting'].isnull(), 'shooting'] = predicted_shooting

MSE: 0.08397599985454487
Validation R² Score: 1.00
Coefficients: [0.4497643  0.05045118 0.19980931 0.20055049 0.04977915 0.04974353]
Intercept: 0.02543768477897146


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [20]:
# Next we shall work on the passing column, we will create a siple linear regression to 
# predict defeding from attacking_crossing, attacking_short_passing, skill_long_passing, skill_curve and mentality_vision
# then use that to extrapolate the passing value in the remaining columns
# the columns were chosen based on research
train_df = df[df['passing'].notnull()]
predict_df = df[df['passing'].isnull()]
X_train = train_df[['attacking_crossing', 
                    'attacking_short_passing', 'skill_long_passing', 'skill_curve', 'mentality_vision']]
y_train = train_df['passing']
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_passing = LinearRegression()
model_passing.fit(X_train1, y_train1)
y_pred = model_passing.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Coefficients and intercept (for transparency)
print("MSE:", mse)
print(f"Validation R² Score: {r2:.2f}")
print("Coefficients:", model_passing.coef_)
print("Intercept:", model_passing.intercept_)

# Retrain on ALL non-null data for better accuracy
model_passing = LinearRegression()
model_passing.fit(X_train, y_train)

X_predict = predict_df[['attacking_crossing', 
                    'attacking_short_passing', 'skill_long_passing', 'skill_curve', 'mentality_vision']]

# Generate predictions
predicted_passing = model_passing.predict(X_predict)

# Fill missing values in original DataFrame
df.loc[df['passing'].isnull(), 'passing'] = predicted_passing

MSE: 0.28591261820520036
Validation R² Score: 1.00
Coefficients: [0.20225138 0.3466598  0.15826024 0.0792165  0.20933122]
Intercept: -0.07957992169257011


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [22]:
# Next we shall work on the dribbling column, we will create a siple linear regression to 
# predict defeding from skill_dribbling, skill_ball_control, movement_agility, movement_balance and mentality_composure
# then use that to extrapolate the dribbling value in the remaining columns
# the columns were chosen based on research
train_df = df[df['dribbling'].notnull()]
predict_df = df[df['dribbling'].isnull()]
X_train = train_df[['skill_dribbling', 
                    'skill_ball_control', 'movement_agility', 'movement_balance', 'mentality_composure']]
y_train = train_df['dribbling']
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_dribbling = LinearRegression()
model_dribbling.fit(X_train1, y_train1)
y_pred = model_dribbling.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Coefficients and intercept (for transparency)
print("MSE:", mse)
print(f"Validation R² Score: {r2:.2f}")
print("Coefficients:", model_dribbling.coef_)
print("Intercept:", model_dribbling.intercept_)

# Retrain on ALL non-null data for better accuracy
model_dribbling = LinearRegression()
model_dribbling.fit(X_train, y_train)

X_predict = predict_df[['skill_dribbling', 
                    'skill_ball_control', 'movement_agility', 'movement_balance', 'mentality_composure']]

# Generate predictions
predicted_dribbling = model_dribbling.predict(X_predict)

# Fill missing values in original DataFrame
df.loc[df['dribbling'].isnull(), 'dribbling'] = predicted_dribbling

MSE: 0.15668941979195103
Validation R² Score: 1.00
Coefficients: [0.44577261 0.31888253 0.09848402 0.04786123 0.07335433]
Intercept: 1.0240441183197575


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [24]:
# Next we shall work on the physic column, we will create a siple linear regression to 
# predict defeding from power_strength, power_stamina, power_jumping, mentality_aggression
# then use that to extrapolate the dribbling value in the remaining columns
# the columns were chosen based on research
train_df = df[df['physic'].notnull()]
predict_df = df[df['physic'].isnull()]
X_train = train_df[['power_strength', 
                    'power_stamina', 'power_jumping', 'mentality_aggression']]
y_train = train_df['physic']
X_train1, X_test, y_train1, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_physic = LinearRegression()
model_physic.fit(X_train1, y_train1)
y_pred = model_physic.predict(X_test)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# Coefficients and intercept (for transparency)
print("MSE:", mse)
print(f"Validation R² Score: {r2:.2f}")
print("Coefficients:", model_physic.coef_)
print("Intercept:", model_physic.intercept_)

# Retrain on ALL non-null data for better accuracy
model_physic = LinearRegression()
model_physic.fit(X_train, y_train)

X_predict = predict_df[['power_strength', 
                    'power_stamina', 'power_jumping', 'mentality_aggression']]

# Generate predictions
predicted_physic = model_physic.predict(X_predict)

# Fill missing values in original DataFrame
df.loc[df['physic'].isnull(), 'physic'] = predicted_physic

MSE: 0.0839021761679498
Validation R² Score: 1.00
Coefficients: [0.49995615 0.25045729 0.05016994 0.19996416]
Intercept: -0.010730507217317609


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [26]:
# Next we shall create a goalkeeping column, we will create a siple linear regression to 
# predict defeding from power_strength, power_stamina, power_jumping, mentality_aggression
# then use that to extrapolate the dribbling value in the remaining columns
# the columns were chosen based on research
df['goalkeeping'] = df[['goalkeeping_diving', 
                      'goalkeeping_handling', 
                      'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes']].fillna(0).mean(axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   short_name                   15333 non-null  object 
 3   long_name                    15333 non-null  object 
 4   player_positions             15333 non-null  object 
 5   overall                      15333 non-null  int64  
 6   potential                    15333 non-null  int64  
 7   value_eur                    15333 non-null  float64
 8   wage_eur                     15333 non-null  float64
 9   age                          15333 non-null  int64  
 10  dob                          15333 non-null  object 
 11  height_cm                    15333 non-null  int64  
 12  weight_kg                    15333 non-null  int64  
 13  club_team_id         

In [27]:
# Removing columns that are no longer relevant:
df = df.drop('short_name', axis=1) # not relevant to predicting
df = df.drop('long_name', axis=1) # not relevant to predicting
df = df.drop('dob', axis=1) # info is already in age column
df = df.drop('club_name', axis=1) # info already in club_id
df = df.drop('league_name', axis=1) # info already in league level

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 71 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   15333 non-null  int64  
 1   id                           15333 non-null  int64  
 2   player_positions             15333 non-null  object 
 3   overall                      15333 non-null  int64  
 4   potential                    15333 non-null  int64  
 5   value_eur                    15333 non-null  float64
 6   wage_eur                     15333 non-null  float64
 7   age                          15333 non-null  int64  
 8   height_cm                    15333 non-null  int64  
 9   weight_kg                    15333 non-null  int64  
 10  club_team_id                 15333 non-null  float64
 11  league_level                 15333 non-null  float64
 12  club_position                15333 non-null  int64  
 13  club_jersey_number   

In [28]:
# Converting this column club_contract_valid_until into number of years left column

In [29]:
df['contract_years_left'] = 2021-df['club_contract_valid_until'].apply(
    lambda x: max(0, x - 2021) if pd.notnull(x) else None
)

In [30]:
# Removing columns that are no longer relevant:
df = df.drop('club_contract_valid_until', axis=1) # info already in club_contract_valid_until
df = df.drop('nationality_name', axis=1) # info already in nationality_id
df = df.drop('body_type', axis=1) # info is already in height and weight columns
df = df.drop(df.columns[27:62], axis=1) # info already in other attributes



df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                15333 non-null  int64  
 1   id                        15333 non-null  int64  
 2   player_positions          15333 non-null  object 
 3   overall                   15333 non-null  int64  
 4   potential                 15333 non-null  int64  
 5   value_eur                 15333 non-null  float64
 6   wage_eur                  15333 non-null  float64
 7   age                       15333 non-null  int64  
 8   height_cm                 15333 non-null  int64  
 9   weight_kg                 15333 non-null  int64  
 10  club_team_id              15333 non-null  float64
 11  league_level              15333 non-null  float64
 12  club_position             15333 non-null  int64  
 13  club_jersey_number        15333 non-null  float64
 14  nationality

In [31]:
# converting preferred foot to numeric where left is -1 right is 1
df['preferred_foot'] = df['preferred_foot'].map({'Left': -1, 'Right': 1}).fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                15333 non-null  int64  
 1   id                        15333 non-null  int64  
 2   player_positions          15333 non-null  object 
 3   overall                   15333 non-null  int64  
 4   potential                 15333 non-null  int64  
 5   value_eur                 15333 non-null  float64
 6   wage_eur                  15333 non-null  float64
 7   age                       15333 non-null  int64  
 8   height_cm                 15333 non-null  int64  
 9   weight_kg                 15333 non-null  int64  
 10  club_team_id              15333 non-null  float64
 11  league_level              15333 non-null  float64
 12  club_position             15333 non-null  int64  
 13  club_jersey_number        15333 non-null  float64
 14  nationality

In [32]:
# Splitting the work rate column and making it numeric

# Mapping dictionary
mapping = {'High': 3, 'Medium': 2, 'Low': 1}

# Split the 'work_rate' column into two new columns
work_rate_split = df['work_rate'].str.split('/', expand=True)

# Map the string values to numbers
df['work_rate1'] = work_rate_split[0].map(mapping).astype(int)
df['work_rate2'] = work_rate_split[1].map(mapping).astype(int)

df = df.drop('work_rate', axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                15333 non-null  int64  
 1   id                        15333 non-null  int64  
 2   player_positions          15333 non-null  object 
 3   overall                   15333 non-null  int64  
 4   potential                 15333 non-null  int64  
 5   value_eur                 15333 non-null  float64
 6   wage_eur                  15333 non-null  float64
 7   age                       15333 non-null  int64  
 8   height_cm                 15333 non-null  int64  
 9   weight_kg                 15333 non-null  int64  
 10  club_team_id              15333 non-null  float64
 11  league_level              15333 non-null  float64
 12  club_position             15333 non-null  int64  
 13  club_jersey_number        15333 non-null  float64
 14  nationality

In [33]:
df['preferred_foot']

0        1
1        1
2       -1
3        1
4        1
        ..
15386    1
15387    1
15388    1
15389    1
15390    1
Name: preferred_foot, Length: 15333, dtype: int64

In [34]:
# Next we will handle the position column we will only look at the first position in that column
# We will split that column into is_goalkeeper, is_midfielder, is_defender and is_attacker

df['first_position'] = df['player_positions'].str.split(',').str[0].str.strip()

# Function to classify positions
def classify_position(pos):
    if pd.isnull(pos):
        return (0, 0, 0, 0)
    last_char = pos[-1].upper()
    if last_char == 'K':
        return (1, 0, 0, 0)  # Goalkeeper
    elif last_char == 'M':
        return (0, 1, 0, 0)  # Midfielder
    elif last_char == 'B':
        return (0, 0, 1, 0)  # Defender
    else:
        return (0, 0, 0, 1)  # Attacker

# Create classification columns
classification = df['first_position'].apply(classify_position)
df[['is_goalkeeper', 'is_midfielder', 'is_defender', 'is_attacker']] = pd.DataFrame(classification.tolist(), index=df.index)

# Calculate position proportions
position_proportions = pd.DataFrame({
    'Position': ['Goalkeeper', 'Midfielder', 'Defender', 'Attacker'],
    'Proportion': [
        df['is_goalkeeper'].mean(),
        df['is_midfielder'].mean(),
        df['is_defender'].mean(),
        df['is_attacker'].mean()
    ]
})

print("\nPosition Proportions:")
print(position_proportions.to_string(index=False))


Position Proportions:
  Position  Proportion
Goalkeeper    0.112372
Midfielder    0.362812
  Defender    0.331964
  Attacker    0.192852


In [35]:
df = df.drop('player_positions', axis=1)
df = df.drop('first_position', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                15333 non-null  int64  
 1   id                        15333 non-null  int64  
 2   overall                   15333 non-null  int64  
 3   potential                 15333 non-null  int64  
 4   value_eur                 15333 non-null  float64
 5   wage_eur                  15333 non-null  float64
 6   age                       15333 non-null  int64  
 7   height_cm                 15333 non-null  int64  
 8   weight_kg                 15333 non-null  int64  
 9   club_team_id              15333 non-null  float64
 10  league_level              15333 non-null  float64
 11  club_position             15333 non-null  int64  
 12  club_jersey_number        15333 non-null  float64
 13  nationality_id            15333 non-null  int64  
 14  preferred_f

# Now we can start building models
We have a few models in mind
First we need to build a model for those with release clause and those without
Then within those 2 domains we need to build a model for each position (goalkeeper, midfielding, defender and attacker)

# First creating models for those with release clauses

In [36]:
df_with_clauses = df.dropna(subset=['release_clause_eur'])
df_with_clauses.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14457 entries, 0 to 15390
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                14457 non-null  int64  
 1   id                        14457 non-null  int64  
 2   overall                   14457 non-null  int64  
 3   potential                 14457 non-null  int64  
 4   value_eur                 14457 non-null  float64
 5   wage_eur                  14457 non-null  float64
 6   age                       14457 non-null  int64  
 7   height_cm                 14457 non-null  int64  
 8   weight_kg                 14457 non-null  int64  
 9   club_team_id              14457 non-null  float64
 10  league_level              14457 non-null  float64
 11  club_position             14457 non-null  int64  
 12  club_jersey_number        14457 non-null  float64
 13  nationality_id            14457 non-null  int64  
 14  preferred_f

## Now building a linear regression to find midfielder value

In [37]:
y = df_with_clauses['value_eur']
X = df_with_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Scale the selected columns
X_scaled_part = pd.DataFrame(
    scaler.fit_transform(X[cols_to_scale]),
    columns=cols_to_scale,
    index=X.index
)

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_midfielder']
)

# Check proportions in training and testing sets
train_prop = X_train['is_midfielder'].value_counts(normalize=True)
test_prop = X_test['is_midfielder'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_midfielder
0    0.636835
1    0.363165
Name: proportion, dtype: float64

Testing Proportions:
 is_midfielder
0    0.636929
1    0.363071
Name: proportion, dtype: float64


In [38]:

param_grid = {
    'alpha': [0.000000001, 0.00000001, 0.0000001, 0.000001, 0.00001 ,0.0001, 0.001, 0.01, 0.1]
}

ridge = Ridge(alpha=1.0)
grid = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)
print("Best R²:", grid.best_score_)

/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.05988e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.10533e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.90224e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.35165e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=

Best parameters: {'alpha': 0.1}
Best R²: 0.99146713653432


/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.07131e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.11832e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=4.90223e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/solaiymeyapan/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=5.35748e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=

In [39]:
model_midfielder_with_release_clause =LinearRegression()
model_midfielder_with_release_clause.fit(X_train, y_train)

predictions = model_midfielder_with_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 489668730673.461
RMSE: 699763.3390464672
R²: 0.9923592979088837


# Linear regression to find defender value

In [40]:
y = df_with_clauses['value_eur']
X = df_with_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_defender']
)

# Check proportions in training and testing sets
train_prop = X_train['is_defender'].value_counts(normalize=True)
test_prop = X_test['is_defender'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_defender
0    0.66537
1    0.33463
Name: proportion, dtype: float64

Testing Proportions:
 is_defender
0    0.665284
1    0.334716
Name: proportion, dtype: float64


In [41]:
model_defender_with_release_clause =LinearRegression()
model_defender_with_release_clause.fit(X_train, y_train)

predictions = model_defender_with_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 484776500478.98376
RMSE: 696258.9320640588
R²: 0.9917880748841776


# Linear regression to find attacker value

In [42]:
y = df_with_clauses['value_eur']
X = df_with_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]


# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_attacker']
)

# Check proportions in training and testing sets
train_prop = X_train['is_attacker'].value_counts(normalize=True)
test_prop = X_test['is_attacker'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_attacker
0    0.812192
1    0.187808
Name: proportion, dtype: float64

Testing Proportions:
 is_attacker
0    0.812241
1    0.187759
Name: proportion, dtype: float64


In [43]:
model_attacker_with_release_clause =LinearRegression()
model_attacker_with_release_clause.fit(X_train, y_train)

predictions = model_attacker_with_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 512718038718.25354
RMSE: 716043.3218166716
R²: 0.9902987780630963


# Linear regression to find goalkeeper value

In [44]:
y = df_with_clauses['value_eur']
X = df_with_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]


# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_goalkeeper']
)

# Check proportions in training and testing sets
train_prop = X_train['is_goalkeeper'].value_counts(normalize=True)
test_prop = X_test['is_goalkeeper'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_goalkeeper
0    0.885603
1    0.114397
Name: proportion, dtype: float64

Testing Proportions:
 is_goalkeeper
0    0.885546
1    0.114454
Name: proportion, dtype: float64


In [45]:
model_goalkeeper_with_release_clause =LinearRegression()
model_goalkeeper_with_release_clause.fit(X_train, y_train)

predictions = model_goalkeeper_with_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 482255217748.3081
RMSE: 694445.9789993085
R²: 0.990201141055217


# Now creating models for those without release clauses

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                15333 non-null  int64  
 1   id                        15333 non-null  int64  
 2   overall                   15333 non-null  int64  
 3   potential                 15333 non-null  int64  
 4   value_eur                 15333 non-null  float64
 5   wage_eur                  15333 non-null  float64
 6   age                       15333 non-null  int64  
 7   height_cm                 15333 non-null  int64  
 8   weight_kg                 15333 non-null  int64  
 9   club_team_id              15333 non-null  float64
 10  league_level              15333 non-null  float64
 11  club_position             15333 non-null  int64  
 12  club_jersey_number        15333 non-null  float64
 13  nationality_id            15333 non-null  int64  
 14  preferred_f

In [47]:
df_without_clauses = df.drop('release_clause_eur', axis=1)
df_without_clauses.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15333 entries, 0 to 15390
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                15333 non-null  int64  
 1   id                        15333 non-null  int64  
 2   overall                   15333 non-null  int64  
 3   potential                 15333 non-null  int64  
 4   value_eur                 15333 non-null  float64
 5   wage_eur                  15333 non-null  float64
 6   age                       15333 non-null  int64  
 7   height_cm                 15333 non-null  int64  
 8   weight_kg                 15333 non-null  int64  
 9   club_team_id              15333 non-null  float64
 10  league_level              15333 non-null  float64
 11  club_position             15333 non-null  int64  
 12  club_jersey_number        15333 non-null  float64
 13  nationality_id            15333 non-null  int64  
 14  preferred_f

# Linear regression to find goalkeeper value

In [48]:
y = df_without_clauses['value_eur']
X = df_without_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Scale the selected columns
X_scaled_part = pd.DataFrame(
    scaler.fit_transform(X[cols_to_scale]),
    columns=cols_to_scale,
    index=X.index
)

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_goalkeeper']
)

# Check proportions in training and testing sets
train_prop = X_train['is_goalkeeper'].value_counts(normalize=True)
test_prop = X_test['is_goalkeeper'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_goalkeeper
0    0.887657
1    0.112343
Name: proportion, dtype: float64

Testing Proportions:
 is_goalkeeper
0    0.887512
1    0.112488
Name: proportion, dtype: float64


In [49]:
model_goalkeeper_without_release_clause =LinearRegression()
model_goalkeeper_without_release_clause.fit(X_train, y_train)

predictions = model_goalkeeper_without_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 12785644354749.518
RMSE: 3575701.9387456663
R²: 0.7390029678027568


# Linear regression to find attacker value

In [50]:
y = df_without_clauses['value_eur']
X = df_without_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Scale the selected columns
X_scaled_part = pd.DataFrame(
    scaler.fit_transform(X[cols_to_scale]),
    columns=cols_to_scale,
    index=X.index
)

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_attacker']
)

# Check proportions in training and testing sets
train_prop = X_train['is_attacker'].value_counts(normalize=True)
test_prop = X_test['is_attacker'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_attacker
0    0.807109
1    0.192891
Name: proportion, dtype: float64

Testing Proportions:
 is_attacker
0    0.807304
1    0.192696
Name: proportion, dtype: float64


In [51]:
model_attacker_without_release_clause =LinearRegression()
model_attacker_without_release_clause.fit(X_train, y_train)

predictions = model_attacker_without_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 15133897166376.008
RMSE: 3890230.9913906152
R²: 0.7692761007256133


# Linear regression to find defender value

In [52]:
y = df_without_clauses['value_eur']
X = df_without_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Scale the selected columns
X_scaled_part = pd.DataFrame(
    scaler.fit_transform(X[cols_to_scale]),
    columns=cols_to_scale,
    index=X.index
)

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_defender']
)

# Check proportions in training and testing sets
train_prop = X_train['is_defender'].value_counts(normalize=True)
test_prop = X_test['is_defender'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_defender
0    0.668025
1    0.331975
Name: proportion, dtype: float64

Testing Proportions:
 is_defender
0    0.66808
1    0.33192
Name: proportion, dtype: float64


In [53]:
model_defender_without_release_clause =LinearRegression()
model_defender_without_release_clause.fit(X_train, y_train)

predictions = model_defender_without_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 9719793922338.488
RMSE: 3117658.403728428
R²: 0.7411595149674886


# Linear regression to find midfielder value

In [54]:
y = df_without_clauses['value_eur']
X = df_without_clauses.drop('value_eur', axis=1)

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in X.columns if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Scale the selected columns
X_scaled_part = pd.DataFrame(
    scaler.fit_transform(X[cols_to_scale]),
    columns=cols_to_scale,
    index=X.index
)

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, X[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

# Perform train-test split with stratification on 'is_midfielder'
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X_final['is_midfielder']
)

# Check proportions in training and testing sets
train_prop = X_train['is_midfielder'].value_counts(normalize=True)
test_prop = X_test['is_midfielder'].value_counts(normalize=True)

print("Training Proportions:\n", train_prop)
print("\nTesting Proportions:\n", test_prop)

Training Proportions:
 is_midfielder
0    0.637209
1    0.362791
Name: proportion, dtype: float64

Testing Proportions:
 is_midfielder
0    0.637105
1    0.362895
Name: proportion, dtype: float64


In [55]:
model_midfielder_without_release_clause =LinearRegression()
model_midfielder_without_release_clause.fit(X_train, y_train)

predictions = model_midfielder_without_release_clause.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")

MSE: 16735923274225.795
RMSE: 4090956.2786988807
R²: 0.6991380004580612


In [56]:
# Currently we have the following models

In [57]:
'''
model_midfielder_with_release_clause
model_midfielder_without_release_clause
model_attacker_with_release_clause
model_attacker_without_release_clause
model_defender_with_release_clause
model_defender_without_release_clause
model_goalkeeper_with_release_clause
model_goalkeeper_without_release_clause
'''

'\nmodel_midfielder_with_release_clause\nmodel_midfielder_without_release_clause\nmodel_attacker_with_release_clause\nmodel_attacker_without_release_clause\nmodel_defender_with_release_clause\nmodel_defender_without_release_clause\nmodel_goalkeeper_with_release_clause\nmodel_goalkeeper_without_release_clause\n'

# Now we shall import the data from the test dataset, then we shall do the same handling of the data and then run the appropriate model on the each row of data

In [58]:
df_test = pd.read_csv("./test.csv")
df_test.head()

,Unnamed: 0,id,short_name,long_name,player_positions,overall,potential,wage_eur,age,dob,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,0,233938,D. Kasumu,David Ayomide Kasumu,CM,63,73,1927.187704,21,1999-10-05,...,61,63,61,58,12,10,11,14,13,NaN
1,1,261545,B. Villacis,Brando Enrique Villacis Vega,CB,57,66,482.073719,22,1998-09-26,...,39,54,56,54,15,11,10,8,7,NaN
2,2,260387,J. Assi,Jean-Aniel Assi,"ST, RW",52,75,471.659254,16,2004-08-12,...,45,22,23,28,11,13,6,7,13,NaN
3,3,241960,T. Stavitski,Timo Stavitski,RW,63,71,946.789206,21,1999-07-17,...,53,17,35,31,14,12,12,9,10,NaN
4,4,234645,N. McGinley,Nathan McGinley,"LB, CB",63,64,954.732171,24,1996-09-15,...,56,61,62,60,13,14,12,9,6,NaN


In [59]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [60]:
# Next we will handle the 'club_loaned_from' column, we will convert this to a
# 'is_on_loan' which is 0 for no and 1 for yes
df_test['is_on_loan'] = df_test['club_loaned_from'].notnull().astype(int)

# Drop the club_loaned_from column
df_test = df_test.drop('club_loaned_from', axis=1)

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [61]:
# Next we will do the same for the national team where if there is a national jersey number
# we set that they have played for the national team and if no number exists then they did not play
df_test['played_for_nation'] = df_test['nation_jersey_number'].notnull().astype(int)

# Drop the club_loaned_from column
df_test = df_test.drop('nation_jersey_number', axis=1)
df_test = df_test.drop('nation_position', axis=1)

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [62]:
# Next we shall fix the club_position column, it has values such as RES, SUB and various positions
# We shall change it to be RES = 3, SUB  = 2 and the rest = 1 this is because reserves are worst then subs are second best
# and the remaining are started which are the best since there is meaning in the number differences this is best
mapping = {'RES': 3, 'SUB': 2}
df_test['club_position'] = df_test['club_position'].map(mapping).fillna(1).astype(int)

In [63]:
# Next is the club_joined column we will find the difference and use that to get the number
# of years at club and for those that are null it means they were at the club from the
# start, so for this we will take the max of the database to get the longest number of years someone
# has been at a club and use this for all 

# Ensure the column is in datetime format
df_test['club_joined'] = pd.to_datetime(df_test['club_joined'], errors='coerce')

# Extract the year (will be NaN if club_joined was null or invalid)
df_test['joined_year'] = df_test['club_joined'].dt.year

# Calculate the number of years at club
df_test['number_of_years_at_club'] = 2021 - df_test['joined_year']

# Fill nulls with max_years
df_test['number_of_years_at_club'].fillna(19, inplace=True) # in this case we fill 19 since that was the max in the training dataset

# Drop the helper column and original column
df_test.drop('joined_year', axis=1, inplace=True)
df_test.drop('club_joined', axis=1, inplace=True)

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [64]:
# We will convert player tags and traits into have traits and tags columns
df_test['has_tags'] = df_test['player_tags'].notnull().astype(int)
df_test['has_traits'] = df_test['player_traits'].notnull().astype(int)

df_test.drop('player_tags', axis=1, inplace=True)
df_test.drop('player_traits', axis=1, inplace=True)

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [65]:
# Identify rows where 'defending' is null
missing_mask = df_test['defending'].isnull()

# Extract the required features for prediction
X_missing = df_test.loc[missing_mask, [
    'defending_marking_awareness',
    'defending_standing_tackle', 
    'defending_sliding_tackle',
    'mentality_interceptions'
]]

# Predict missing defending values
predicted_defending = model_defense.predict(X_missing)

# Fill the null values in original DataFrame
df_test.loc[missing_mask, 'defending'] = predicted_defending

# Show results
print(f"Predicted {len(predicted_defending)} missing 'defending' values")
print("\nPreview of updated rows:")
print(df_test.loc[missing_mask, [
    'defending_marking_awareness',
    'defending_standing_tackle',
    'defending_sliding_tackle', 
    'mentality_interceptions',
    'defending'
]].head())

Predicted 403 missing 'defending' values

Preview of updated rows:
    defending_marking_awareness  defending_standing_tackle  \
22                           12                         13   
48                           17                         11   
54                           15                         15   
67                            8                         11   
87                           18                         23   

    defending_sliding_tackle  mentality_interceptions  defending  
22                        10                       11  15.717812  
48                        17                       17  18.601304  
54                        17                       16  18.936428  
67                        14                       13  14.601880  
87                        18                       24  24.100233  


In [66]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [67]:
# Identify rows where 'pace' is null
missing_mask = df_test['pace'].isnull()

# Extract the required features for prediction
X_missing = df_test.loc[missing_mask, [
    'movement_acceleration',
    'movement_sprint_speed'
]]

# Predict missing pace values
predicted_pace = model_pace.predict(X_missing)

# Fill the null values in original DataFrame
df_test.loc[missing_mask, 'pace'] = predicted_pace

# Show results
print(f"Predicted {len(predicted_pace)} missing 'pace' values")
print("\nPreview of updated rows:")
print(df_test.loc[missing_mask, [
    'movement_acceleration',
    'movement_sprint_speed',
    'pace'
]].head())


Predicted 403 missing 'pace' values

Preview of updated rows:
    movement_acceleration  movement_sprint_speed       pace
22                     30                     31  30.584256
48                     47                     45  45.919819
54                     45                     45  45.024159
67                     18                     23  20.797466
87                     54                     43  47.951421


In [68]:
# Identify rows where 'shooting' is null
missing_mask = df_test['shooting'].isnull()

# Extract the required features for prediction
X_missing = df_test.loc[missing_mask, [
    'attacking_finishing',
    'attacking_volleys',
    'power_shot_power',
    'power_long_shots',
    'mentality_positioning',
    'mentality_penalties'
]]

# Predict missing shooting values
predicted_shooting = model_shooting.predict(X_missing)

# Fill the null values in original DataFrame
df_test.loc[missing_mask, 'shooting'] = predicted_shooting

# Show results
print(f"Predicted {len(predicted_shooting)} missing 'shooting' values")
print("\nPreview of updated rows:")
print(df_test.loc[missing_mask, [
    'attacking_finishing',
    'attacking_volleys',
    'power_shot_power',
    'power_long_shots',
    'mentality_positioning',
    'mentality_penalties',
    'shooting'
]].head())


Predicted 403 missing 'shooting' values

Preview of updated rows:
    attacking_finishing  attacking_volleys  power_shot_power  \
22                    7                  5                48   
48                   14                 17                52   
54                   18                 15                48   
67                    5                  7                44   
87                   14                 20                46   

    power_long_shots  mentality_positioning  mentality_penalties   shooting  
22                 5                      4                   17  15.064680  
48                15                     18                   24  22.668400  
54                14                     15                   17  22.868903  
67                 9                      5                   16  14.268673  
87                15                     18                   15  21.172482  


In [69]:
# Identify rows where 'passing' is null
missing_mask = df_test['passing'].isnull()

# Extract the required features for prediction
X_missing = df_test.loc[missing_mask, [
    'attacking_crossing',
    'attacking_short_passing',
    'skill_long_passing',
    'skill_curve',
    'mentality_vision'
]]

# Predict missing passing values
predicted_passing = model_passing.predict(X_missing)

# Fill the null values in original DataFrame
df_test.loc[missing_mask, 'passing'] = predicted_passing

# Show results
print(f"Predicted {len(predicted_passing)} missing 'passing' values")
print("\nPreview of updated rows:")
print(df_test.loc[missing_mask, [
    'attacking_crossing',
    'attacking_short_passing',
    'skill_long_passing',
    'skill_curve',
    'mentality_vision',
    'passing'
]].head())


Predicted 403 missing 'passing' values

Preview of updated rows:
    attacking_crossing  attacking_short_passing  skill_long_passing  \
22                  10                       45                  56   
48                  18                       37                  44   
54                  14                       17                  19   
67                  14                       22                  21   
87                  33                       42                  44   

    skill_curve  mentality_vision    passing  
22           11                29  33.351250  
48           11                53  35.311822  
54           12                40  20.980911  
67           14                34  21.934160  
87           14                56  40.940130  


In [70]:
# Identify rows where 'dribbling' is null
missing_mask = df_test['dribbling'].isnull()

# Extract the required features for prediction
X_missing = df_test.loc[missing_mask, [
    'skill_dribbling',
    'skill_ball_control',
    'movement_agility',
    'movement_balance',
    'mentality_composure'
]]

# Predict missing dribbling values
predicted_dribbling = model_dribbling.predict(X_missing)

# Fill the null values in original DataFrame
df_test.loc[missing_mask, 'dribbling'] = predicted_dribbling

# Show results
print(f"Predicted {len(predicted_dribbling)} missing 'dribbling' values")
print("\nPreview of updated rows:")
print(df_test.loc[missing_mask, [
    'skill_dribbling',
    'skill_ball_control',
    'movement_agility',
    'movement_balance',
    'mentality_composure',
    'dribbling'
]].head())


Predicted 403 missing 'dribbling' values

Preview of updated rows:
    skill_dribbling  skill_ball_control  movement_agility  movement_balance  \
22               14                  19                31                41   
48               11                  16                59                42   
54               17                  16                33                32   
67                5                  19                39                44   
87               18                  22                61                63   

    mentality_composure  dribbling  
22                   35  20.904982  
48                   47  22.300428  
54                   32  20.836444  
67                   27  17.240467  
87                   62  29.627373  


In [71]:
# Identify rows where 'physic' is null
missing_mask = df_test['physic'].isnull()

# Extract the required features for prediction
X_missing = df_test.loc[missing_mask, [
    'power_strength',
    'power_stamina',
    'power_jumping',
    'mentality_aggression'
]]

# Predict missing physic values
predicted_physic = model_physic.predict(X_missing)

# Fill the null values in original DataFrame
df_test.loc[missing_mask, 'physic'] = predicted_physic

# Show results
print(f"Predicted {len(predicted_physic)} missing 'physic' values")
print("\nPreview of updated rows:")
print(df_test.loc[missing_mask, [
    'power_strength',
    'power_stamina',
    'power_jumping',
    'mentality_aggression',
    'physic'
]].head())


Predicted 403 missing 'physic' values

Preview of updated rows:
    power_strength  power_stamina  power_jumping  mentality_aggression  \
22              60             40             74                    17   
48              62             31             65                    29   
54              56             24             57                    16   
67              48             16             61                    17   
87              57             37             63                    37   

       physic  
22  47.123535  
48  47.819905  
54  40.068152  
67  34.468508  
87  48.321986  


In [72]:
# Next we shall create a goalkeeping column, we will create a siple linear regression to 
# predict defeding from power_strength, power_stamina, power_jumping, mentality_aggression
# then use that to extrapolate the dribbling value in the remaining columns
# the columns were chosen based on research
df_test['goalkeeping'] = df_test[['goalkeeping_diving', 
                      'goalkeeping_handling', 
                      'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes']].fillna(0).mean(axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   short_name                   3848 non-null   object 
 3   long_name                    3848 non-null   object 
 4   player_positions             3848 non-null   object 
 5   overall                      3848 non-null   int64  
 6   potential                    3848 non-null   int64  
 7   wage_eur                     3836 non-null   float64
 8   age                          3848 non-null   int64  
 9   dob                          3848 non-null   object 
 10  height_cm                    3848 non-null   int64  
 11  weight_kg                    3848 non-null   int64  
 12  club_team_id                 3836 non-null   float64
 13  club_name         

In [73]:
# Removing columns that are no longer relevant:
df_test = df_test.drop('short_name', axis=1) # not relevant to predicting
df_test = df_test.drop('long_name', axis=1) # not relevant to predicting
df_test = df_test.drop('dob', axis=1) # info is already in age column
df_test = df_test.drop('club_name', axis=1) # info already in club_id
df_test = df_test.drop('league_name', axis=1) # info already in league level

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 70 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   3848 non-null   int64  
 1   id                           3848 non-null   int64  
 2   player_positions             3848 non-null   object 
 3   overall                      3848 non-null   int64  
 4   potential                    3848 non-null   int64  
 5   wage_eur                     3836 non-null   float64
 6   age                          3848 non-null   int64  
 7   height_cm                    3848 non-null   int64  
 8   weight_kg                    3848 non-null   int64  
 9   club_team_id                 3836 non-null   float64
 10  league_level                 3836 non-null   float64
 11  club_position                3848 non-null   int64  
 12  club_jersey_number           3836 non-null   float64
 13  club_contract_vali

In [74]:
df_test['contract_years_left'] = 2021-df_test['club_contract_valid_until'].apply(
    lambda x: max(0, x - 2021) if pd.notnull(x) else None
)

In [75]:
# Removing columns that are no longer relevant:
df_test = df_test.drop('club_contract_valid_until', axis=1) # info already in club_contract_valid_until
df_test = df_test.drop('nationality_name', axis=1) # info already in nationality_id
df_test = df_test.drop('body_type', axis=1) # info is already in height and weight columns
df_test = df_test.drop(df_test.columns[26:61], axis=1) # info already in other attributes



df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   player_positions          3848 non-null   object 
 3   overall                   3848 non-null   int64  
 4   potential                 3848 non-null   int64  
 5   wage_eur                  3836 non-null   float64
 6   age                       3848 non-null   int64  
 7   height_cm                 3848 non-null   int64  
 8   weight_kg                 3848 non-null   int64  
 9   club_team_id              3836 non-null   float64
 10  league_level              3836 non-null   float64
 11  club_position             3848 non-null   int64  
 12  club_jersey_number        3836 non-null   float64
 13  nationality_id            3848 non-null   int64  
 14  preferre

In [76]:
# converting preferred foot to numeric where left is -1 right is 1
df_test['preferred_foot'] = df_test['preferred_foot'].map({'Left': -1, 'Right': 1}).fillna(0).astype(int)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   player_positions          3848 non-null   object 
 3   overall                   3848 non-null   int64  
 4   potential                 3848 non-null   int64  
 5   wage_eur                  3836 non-null   float64
 6   age                       3848 non-null   int64  
 7   height_cm                 3848 non-null   int64  
 8   weight_kg                 3848 non-null   int64  
 9   club_team_id              3836 non-null   float64
 10  league_level              3836 non-null   float64
 11  club_position             3848 non-null   int64  
 12  club_jersey_number        3836 non-null   float64
 13  nationality_id            3848 non-null   int64  
 14  preferre

In [77]:
# Splitting the work rate column and making it numeric

# Mapping dictionary
mapping = {'High': 3, 'Medium': 2, 'Low': 1}

# Split the 'work_rate' column into two new columns
work_rate_split = df_test['work_rate'].str.split('/', expand=True)

# Map the string values to numbers
df_test['work_rate1'] = work_rate_split[0].map(mapping).astype(int)
df_test['work_rate2'] = work_rate_split[1].map(mapping).astype(int)

df_test = df_test.drop('work_rate', axis=1)

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   player_positions          3848 non-null   object 
 3   overall                   3848 non-null   int64  
 4   potential                 3848 non-null   int64  
 5   wage_eur                  3836 non-null   float64
 6   age                       3848 non-null   int64  
 7   height_cm                 3848 non-null   int64  
 8   weight_kg                 3848 non-null   int64  
 9   club_team_id              3836 non-null   float64
 10  league_level              3836 non-null   float64
 11  club_position             3848 non-null   int64  
 12  club_jersey_number        3836 non-null   float64
 13  nationality_id            3848 non-null   int64  
 14  preferre

In [78]:
# Next we will handle the position column we will only look at the first position in that column
# We will split that column into is_goalkeeper, is_midfielder, is_defender and is_attacker

df_test['first_position'] = df_test['player_positions'].str.split(',').str[0].str.strip()

# Function to classify positions
def classify_position(pos):
    if pd.isnull(pos):
        return (0, 0, 0, 0)
    last_char = pos[-1].upper()
    if last_char == 'K':
        return (1, 0, 0, 0)  # Goalkeeper
    elif last_char == 'M':
        return (0, 1, 0, 0)  # Midfielder
    elif last_char == 'B':
        return (0, 0, 1, 0)  # Defender
    else:
        return (0, 0, 0, 1)  # Attacker

# Create classification columns
classification = df_test['first_position'].apply(classify_position)
df_test[['is_goalkeeper', 'is_midfielder', 'is_defender', 'is_attacker']] = pd.DataFrame(classification.tolist(), index=df_test.index)

# Calculate position proportions
position_proportions = pd.DataFrame({
    'Position': ['Goalkeeper', 'Midfielder', 'Defender', 'Attacker'],
    'Proportion': [
        df['is_goalkeeper'].mean(),
        df['is_midfielder'].mean(),
        df['is_defender'].mean(),
        df['is_attacker'].mean()
    ]
})

print("\nPosition Proportions:")
print(position_proportions.to_string(index=False))


Position Proportions:
  Position  Proportion
Goalkeeper    0.112372
Midfielder    0.362812
  Defender    0.331964
  Attacker    0.192852


In [79]:
df_test = df_test.drop('player_positions', axis=1)
df_test = df_test.drop('first_position', axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   overall                   3848 non-null   int64  
 3   potential                 3848 non-null   int64  
 4   wage_eur                  3836 non-null   float64
 5   age                       3848 non-null   int64  
 6   height_cm                 3848 non-null   int64  
 7   weight_kg                 3848 non-null   int64  
 8   club_team_id              3836 non-null   float64
 9   league_level              3836 non-null   float64
 10  club_position             3848 non-null   int64  
 11  club_jersey_number        3836 non-null   float64
 12  nationality_id            3848 non-null   int64  
 13  preferred_foot            3848 non-null   int64  
 14  weak_foo

In [80]:
df_value_eur = pd.DataFrame({'value_eur': [np.nan] * len(df_test)})

In [81]:
# Scaling

# Columns to exclude from scaling
exclude_cols = ['is_midfielder', 'is_attacker', 'is_goalkeeper', 'is_defender']

# Columns to scale
cols_to_scale = [col for col in df_test.columns if col not in exclude_cols]

# Initialize scaler
scaler = StandardScaler()

# Scale the selected columns using the index from df_test
X_scaled_part = pd.DataFrame(
    scaler.fit_transform(df_test[cols_to_scale]),
    columns=cols_to_scale,
    index=df_test.index
)

# Concatenate scaled and unscaled columns
X_final = pd.concat([X_scaled_part, df_test[exclude_cols]], axis=1)

# Reorder columns to original order
X_final = X_final[X.columns]

In [82]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   overall                   3848 non-null   int64  
 3   potential                 3848 non-null   int64  
 4   wage_eur                  3836 non-null   float64
 5   age                       3848 non-null   int64  
 6   height_cm                 3848 non-null   int64  
 7   weight_kg                 3848 non-null   int64  
 8   club_team_id              3836 non-null   float64
 9   league_level              3836 non-null   float64
 10  club_position             3848 non-null   int64  
 11  club_jersey_number        3836 non-null   float64
 12  nationality_id            3848 non-null   int64  
 13  preferred_foot            3848 non-null   int64  
 14  weak_foo

In [83]:
cols_to_fill = [
    'club_jersey_number',
    'club_team_id',
    'contract_years_left',
    'league_level',
    'wage_eur'
]

df_test[cols_to_fill] = df_test[cols_to_fill].fillna(df_test[cols_to_fill].mean())

In [84]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   overall                   3848 non-null   int64  
 3   potential                 3848 non-null   int64  
 4   wage_eur                  3848 non-null   float64
 5   age                       3848 non-null   int64  
 6   height_cm                 3848 non-null   int64  
 7   weight_kg                 3848 non-null   int64  
 8   club_team_id              3848 non-null   float64
 9   league_level              3848 non-null   float64
 10  club_position             3848 non-null   int64  
 11  club_jersey_number        3848 non-null   float64
 12  nationality_id            3848 non-null   int64  
 13  preferred_foot            3848 non-null   int64  
 14  weak_foo

In [85]:


for i in range(len(df_test)):
    if df_test.loc[i, 'is_goalkeeper'] == 1 and pd.notnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Goalkeeper, Release Clause = {df_test.loc[i, 'release_clause_eur']}")
        df_value_eur['value_eur'][i] = model_goalkeeper_with_release_clause.predict(df_test.iloc[[i]])
    elif df_test.loc[i, 'is_goalkeeper'] == 1 and pd.isnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Goalkeeper, No release clause")
        df_value_eur['value_eur'][i] = model_goalkeeper_without_release_clause.predict(df_test.iloc[[i]].dropna(axis=1))
    elif df_test.loc[i, 'is_defender'] == 1 and pd.notnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Defender, Release Clause = {df_test.loc[i, 'release_clause_eur']}")
        df_value_eur['value_eur'][i] = model_defender_with_release_clause.predict(df_test.iloc[[i]])
    elif df_test.loc[i, 'is_defender'] == 1 and pd.isnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Defender, No release clause")
        df_value_eur['value_eur'][i] = model_defender_without_release_clause.predict(df_test.iloc[[i]].dropna(axis=1))
    elif df_test.loc[i, 'is_midfielder'] == 1 and pd.notnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Midfielder, Release Clause = {df_test.loc[i, 'release_clause_eur']}")
        df_value_eur['value_eur'][i] = model_midfielder_with_release_clause.predict(df_test.iloc[[i]])
    elif df_test.loc[i, 'is_midfielder'] == 1 and pd.isnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Midfielder, No release clause")
        df_value_eur['value_eur'][i] = model_midfielder_without_release_clause.predict(df_test.iloc[[i]].dropna(axis=1))
    elif df_test.loc[i, 'is_attacker'] == 1 and pd.notnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Attacker, Release Clause = {df_test.loc[i, 'release_clause_eur']}")
        df_value_eur['value_eur'][i] = model_attacker_with_release_clause.predict(df_test.iloc[[i]])
    elif df_test.loc[i, 'is_attacker'] == 1 and pd.isnull(df_test.loc[i, 'release_clause_eur']):
        print(f"Row {i}: Position = Attacker, No release clause")
        df_value_eur['value_eur'][i] = model_attacker_without_release_clause.predict(df_test.iloc[[i]].dropna(axis=1))



Row 0: Position = Midfielder, Release Clause = 2100000.0
Row 1: Position = Defender, Release Clause = 823000.0
Row 2: Position = Attacker, Release Clause = 613000.0
Row 3: Position = Attacker, Release Clause = 2100000.0
Row 4: Position = Defender, Release Clause = 1000000.0
Row 5: Position = Defender, Release Clause = 1900000.0
Row 6: Position = Midfielder, No release clause
Row 7: Position = Midfielder, Release Clause = 2300000.0
Row 8: Position = Midfielder, Release Clause = 1600000.0
Row 9: Position = Defender, Release Clause = 979000.0
Row 10: Position = Midfielder, Release Clause = 5300000.0
Row 11: Position = Defender, Release Clause = 1400000.0
Row 12: Position = Attacker, Release Clause = 36100000.0
Row 13: Position = Defender, No release clause
Row 14: Position = Attacker, Release Clause = 7000000.0
Row 15: Position = Defender, Release Clause = 363000.0
Row 16: Position = Midfielder, No release clause
Row 17: Position = Defender, Release Clause = 238000.0
Row 18: Position = De

In [86]:
df_value_eur

,value_eur
0,9.335289e+05
1,3.922770e+05
2,1.490968e+05
3,9.726246e+05
4,6.124333e+05
...,...
3843,6.991846e+05
3844,4.416848e+07
3845,5.348055e+05
3846,7.183587e+05


In [87]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                3848 non-null   int64  
 1   id                        3848 non-null   int64  
 2   overall                   3848 non-null   int64  
 3   potential                 3848 non-null   int64  
 4   wage_eur                  3848 non-null   float64
 5   age                       3848 non-null   int64  
 6   height_cm                 3848 non-null   int64  
 7   weight_kg                 3848 non-null   int64  
 8   club_team_id              3848 non-null   float64
 9   league_level              3848 non-null   float64
 10  club_position             3848 non-null   int64  
 11  club_jersey_number        3848 non-null   float64
 12  nationality_id            3848 non-null   int64  
 13  preferred_foot            3848 non-null   int64  
 14  weak_foo

In [88]:
'''
model_midfielder_with_release_clause
model_midfielder_without_release_clause
model_attacker_with_release_clause
model_attacker_without_release_clause
model_defender_with_release_clause
model_defender_without_release_clause
model_goalkeeper_with_release_clause
model_goalkeeper_without_release_clause
'''

'\nmodel_midfielder_with_release_clause\nmodel_midfielder_without_release_clause\nmodel_attacker_with_release_clause\nmodel_attacker_without_release_clause\nmodel_defender_with_release_clause\nmodel_defender_without_release_clause\nmodel_goalkeeper_with_release_clause\nmodel_goalkeeper_without_release_clause\n'